# 03 - Algorithms

In this notebook you can find simple examples that showcase how to predict the best future portfolio values according to some heuristic.

You can create your own algorithms by creating a Python module in `modules/experts/`. You have to follow the format of `modules/experts/Sharpe.py`.

In [1]:
import sys
sys.path.insert(0, '../')
%reload_ext autoreload
%autoreload 2

## Start server

The `Server` class is the interface to perform HTTPS requests to acquire data regarding instruments, like stocks.

In [2]:
import time
import logging

import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

from modules.server import Server
from modules.instruments import Portfolio

In [3]:
srv = Server()
srv.start()

# the following silences the logging output
# logging.getLogger('ibkr-algotrading').setLevel(logging.ERROR)

Once started, we can request information on any stock we want simply by accesing items in `srv`. These will be cached, so only the first access will have significant delay.

## Create Portfolio

we can now request some stocks and create a portfolio with them, adding the portfolio to the server so it knows about its existance

In [4]:
STOCKS = ['SPY', 'XOM', 'MSFT', 'AAPL', 'TSLA', 'IBM', 'AMZN', 'GOOG', 'FB', 'ADBE']
# historical data in a 1 year period in 1 day intervals
stocks = srv[STOCKS, '1y', '1d']

# wealth of 100 USD
portfolio = Portfolio(wealth=100, stocks=stocks)

srv.add_portfolio(portfolio)

[2021-09-09 18:38:03,222] I T140699101365824 server.py:467: Started StockCache thread for bar "1d". 
		Next data update will be in  26516.78s.
[2021-09-09 18:38:03,275] I T140699109758528 server.py:449: Loaded "Stock("SPY", conid=756733)" from StockCache's storage.
[2021-09-09 18:38:03,330] I T140699092973120 server.py:449: Loaded "Stock("XOM", conid=13977)" from StockCache's storage.
[2021-09-09 18:38:03,362] I T140699084580416 server.py:449: Loaded "Stock("MSFT", conid=272093)" from StockCache's storage.
[2021-09-09 18:38:03,383] I T140699076187712 server.py:449: Loaded "Stock("AAPL", conid=265598)" from StockCache's storage.
[2021-09-09 18:38:03,403] I T140699067795008 server.py:449: Loaded "Stock("TSLA", conid=76792991)" from StockCache's storage.
[2021-09-09 18:38:03,721] I T140699109758528 server.py:623: Updated: Stock("SPY", conid=756733) - bar: 1d - added period: 1y
[2021-09-09 18:38:03,733] I T140699109758528 server.py:449: Loaded "Stock("IBM", conid=8314)" from StockCache's s

## Experts

Experts are sub-algorithms that provide a prediction to how the wealth of the portfolio should be allocated.

We will use an expert that predicts the allocation that maximizes the sharpe ratio of the portfolio.

In [6]:
from modules import experts

# some experts may need initialisation
experts.Sharpe.init()

# predict for tomorrow
date = pd.Timestamp.today() + pd.Timedelta(days=1)
print('Predicting for date:', date)

weights = experts.Sharpe.predict(pf=portfolio, date=date, srv=srv)

[2021-09-09 18:38:49,986] I T140700537546560 Sharpe.py:70: Starting Sharpe ratio optimizer for dates: 2020-09-11 13:30:00 -> 2021-09-09 18:38:49.978662


Predicting for date: 2021-09-10 18:38:49.978662
Done: Stock("SPY", conid=756733)
Done: Stock("XOM", conid=13977)
Done: Stock("MSFT", conid=272093)
Done: Stock("AAPL", conid=265598)
Done: Stock("TSLA", conid=76792991)
Done: Stock("IBM", conid=8314)
Done: Stock("AMZN", conid=3691937)
Done: Stock("GOOG", conid=208813720)


[2021-09-09 18:38:53,182] I T140700537546560 Sharpe.py:98: Retrieving stock values took 3.194s
[2021-09-09 18:38:53,184] I T140700537546560 Sharpe.py:106: sharpe:  0.33 
		weights: [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
[2021-09-09 18:38:53,186] I T140700537546560 Sharpe.py:106: sharpe:  0.33 
		weights: [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
[2021-09-09 18:38:53,186] I T140700537546560 Sharpe.py:106: sharpe:  0.33 
		weights: [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
[2021-09-09 18:38:53,187] I T140700537546560 Sharpe.py:106: sharpe:  0.33 
		weights: [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
[2021-09-09 18:38:53,189] I T140700537546560 Sharpe.py:106: sharpe:  0.33 
		weights: [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
[2021-09-09 18:38:53,190] I T140700537546560 Sharpe.py:106: sharpe:  0.33 
		weights: [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
[2021-09-09 18:38:53,190] I T140700537546560 Sharpe.py:106: sharpe:  0.33 
		weight

Done: Stock("FB", conid=107113386)
